In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/'

In [3]:
c = requests.get(url).content
soup = BeautifulSoup(c)

In [4]:
aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}) 
len(aluguel_tag)

36

In [5]:
paginas = ['']
for i in range(2,40):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)

?pagina=2
?pagina=3
?pagina=4
?pagina=5
?pagina=6
?pagina=7
?pagina=8
?pagina=9
?pagina=10
?pagina=11
?pagina=12
?pagina=13
?pagina=14
?pagina=15
?pagina=16
?pagina=17
?pagina=18
?pagina=19
?pagina=20
?pagina=21
?pagina=22
?pagina=23
?pagina=24
?pagina=25
?pagina=26
?pagina=27
?pagina=28
?pagina=29
?pagina=30
?pagina=31
?pagina=32
?pagina=33
?pagina=34
?pagina=35
?pagina=36
?pagina=37
?pagina=38
?pagina=39


In [6]:
import pandas as pd
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista,
            'vaga':vaga_lista,
            'area':area_lista}
    df = pd.DataFrame(data)
    return df

In [7]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    
    aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}) 
    aluguel_lista = [p.text for p in aluguel_tag] 
    #print(len(aluguel_lista)) 
    
    quartos_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'}) 
    quartos_lista = [en.text for en in quartos_tag] 
    #print(len(quartos_lista)) 
    
    endereco_tag = soup.find_all('span', {'class': 'property-card__address'}) 
    endereco_lista = [fg.text for fg in endereco_tag] 
    #print(len(endereco_lista)) 
    
    banheiro_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}) 
    banheiro_lista = [yp.text for yp in banheiro_tag] 
    #print(len(banheiro_lista))

    vaga_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'}) 
    vaga_lista = [yp.text for yp in vaga_tag] 

    area_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-area'}) 
    area_lista = [yp.text for yp in area_tag] 


    ### Vamos ter criar uma função para transformar listas em df
    df=converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista)
    
    return df

In [8]:
URL = 'https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/{}'

In [9]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)

df_final = pd.concat(df_lista)

Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=2
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=3
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=4
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=5
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=6
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=7
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=8
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=9
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-grande/?pagina=10
Coletando URL: https://www.vivareal.com.br/aluguel/mato-grosso-do-sul/campo-gr

In [10]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,area
0,R$ 400 /Mês,1 Quarto,"Avenida Tamandaré, 3192 - Vila Nasser, Campo G...",1 Banheiro,-- Vaga,17 m²
1,R$ 5.900 /Mês,3 Quartos,"Avenida Bom Pastor, 1473 - Vila Vilas Boas, Ca...",3 Banheiros,1 Vaga,210 m²
2,R$ 60.000 /Mês R$ 8.000.000,-- Quarto,"Rua Marechal Rondon, 1518 - Centro, Campo Gran...",-- Banheiro,-- Vaga,2540 m²
3,R$ 1.400 /Mês R$ 310.000,3 Quartos,"Rua Doutor Werneck - UFMS, Campo Grande - MS",2 Banheiros,1 Vaga,66 m²
4,R$ 3.000 /Mês R$ 650.000,2 Quartos,"Avenida Três Barras, 859 - Vila Vilas Boas, Ca...",1 Banheiro,2 Vagas,150 m²


In [11]:
df_final.to_csv('aluguel.csv')

In [12]:
df_final['bairro'] = df_final.endereco.str.split('\s-').str[1]
df_final['rua'] = df_final.endereco.str.split('\s-').str[0]
df_final['bairro'] = df_final.bairro.str.split(',').str[0]

df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,area,bairro,rua
0,R$ 400 /Mês,1 Quarto,"Avenida Tamandaré, 3192 - Vila Nasser, Campo G...",1 Banheiro,-- Vaga,17 m²,Vila Nasser,"Avenida Tamandaré, 3192"
1,R$ 5.900 /Mês,3 Quartos,"Avenida Bom Pastor, 1473 - Vila Vilas Boas, Ca...",3 Banheiros,1 Vaga,210 m²,Vila Vilas Boas,"Avenida Bom Pastor, 1473"
2,R$ 60.000 /Mês R$ 8.000.000,-- Quarto,"Rua Marechal Rondon, 1518 - Centro, Campo Gran...",-- Banheiro,-- Vaga,2540 m²,Centro,"Rua Marechal Rondon, 1518"
3,R$ 1.400 /Mês R$ 310.000,3 Quartos,"Rua Doutor Werneck - UFMS, Campo Grande - MS",2 Banheiros,1 Vaga,66 m²,UFMS,Rua Doutor Werneck
4,R$ 3.000 /Mês R$ 650.000,2 Quartos,"Avenida Três Barras, 859 - Vila Vilas Boas, Ca...",1 Banheiro,2 Vagas,150 m²,Vila Vilas Boas,"Avenida Três Barras, 859"


In [13]:
import pandas as pd

In [14]:
aluguel = pd.read_csv('aluguel.csv',dtype = str)

In [15]:
aluguel.head

<bound method NDFrame.head of      Unnamed: 0  ...           area
0             0  ...      17   m²  
1             1  ...     210   m²  
2             2  ...    2540   m²  
3             3  ...      66   m²  
4             4  ...     150   m²  
...         ...  ...            ...
1399         31  ...     223   m²  
1400         32  ...     114   m²  
1401         33  ...     135   m²  
1402         34  ...      90   m²  
1403         35  ...      62   m²  

[1404 rows x 7 columns]>

In [16]:
aluguel.shape

(1404, 7)

In [17]:
aluguel.aluguel

0                              R$ 400 /Mês  
1                            R$ 5.900 /Mês  
2          R$ 60.000 /Mês    R$ 8.000.000   
3             R$ 1.400 /Mês    R$ 310.000   
4             R$ 3.000 /Mês    R$ 650.000   
                        ...                 
1399                         R$ 5.500 /Mês  
1400                         R$ 1.500 /Mês  
1401                         R$ 2.800 /Mês  
1402                         R$ 1.300 /Mês  
1403                         R$ 2.000 /Mês  
Name: aluguel, Length: 1404, dtype: object

In [18]:
aluguel.head(2)

,Unnamed: 0,aluguel,quartos,endereco,banheiro,vaga,area
0,0,R$ 400 /Mês,1 Quarto,"Avenida Tamandaré, 3192 - Vila Nasser, Campo G...",1 Banheiro,-- Vaga,17 m²
1,1,R$ 5.900 /Mês,3 Quartos,"Avenida Bom Pastor, 1473 - Vila Vilas Boas, Ca...",3 Banheiros,1 Vaga,210 m²


In [19]:
aluguel['aluguel'] = aluguel['aluguel'].replace('Sob Consulta','',regex = True) 

In [20]:
aluguel['quartos'] = aluguel['quartos'].replace('\--',0,regex = True)

In [21]:
aluguel = aluguel.drop(['Unnamed: 0'],axis=1)

In [23]:
aluguel['bairro'] = aluguel.endereco.str.split('\s-').str[1]
aluguel['rua'] = aluguel.endereco.str.split('\s-').str[0]
aluguel['bairro'] = aluguel.bairro.str.split(',').str[0]
aluguel['vaga'] = aluguel.vaga.str.extract('(.*) Vaga')


In [24]:
aluguel.head(50)

,aluguel,quartos,endereco,banheiro,vaga,area,bairro,rua
0,R$ 400 /Mês,1 Quarto,"Avenida Tamandaré, 3192 - Vila Nasser, Campo G...",1 Banheiro,NaN,17 m²,Vila Nasser,"Avenida Tamandaré, 3192"
1,R$ 5.900 /Mês,3 Quartos,"Avenida Bom Pastor, 1473 - Vila Vilas Boas, Ca...",3 Banheiros,NaN,210 m²,Vila Vilas Boas,"Avenida Bom Pastor, 1473"
2,R$ 60.000 /Mês R$ 8.000.000,0,"Rua Marechal Rondon, 1518 - Centro, Campo Gran...",-- Banheiro,NaN,2540 m²,Centro,"Rua Marechal Rondon, 1518"
3,R$ 1.400 /Mês R$ 310.000,3 Quartos,"Rua Doutor Werneck - UFMS, Campo Grande - MS",2 Banheiros,NaN,66 m²,UFMS,Rua Doutor Werneck
4,R$ 3.000 /Mês R$ 650.000,2 Quartos,"Avenida Três Barras, 859 - Vila Vilas Boas, Ca...",1 Banheiro,NaN,150 m²,Vila Vilas Boas,"Avenida Três Barras, 859"
5,R$ 900 /Mês,3 Quartos,"Amambaí, Campo Grande - MS",2 Banheiros,NaN,93 m²,MS,"Amambaí, Campo Grande"
6,R$ 1.500 /Mês,2 Quartos,"Avenida Senador Antonio Mendes Canale, 725 - P...",2 Banheiros,NaN,68 m²,Pioneiros,"Avenida Senador Antonio Mendes Canale, 725"
7,R$ 2.200 /Mês,0,"Rua Quinze de Novembro - Jardim dos Estados, C...",1 Banheiro,NaN,52 m²,Jardim dos Estados,Rua Quinze de Novembro
8,R$ 1.650 /Mês,2 Quartos,"Taquaral Bosque, Campo Grande - MS",1 Banheiro,NaN,65 m²,MS,"Taquaral Bosque, Campo Grande"
9,R$ 9.800 /Mês,0,"Avenida Jamil Nahas, 611 - Polo Empresarial Oe...",4 Banheiros,NaN,1000 m²,Polo Empresarial Oeste,"Avenida Jamil Nahas, 611"


In [25]:
cols_str = aluguel.columns.drop(['endereco','bairro','rua'])

In [26]:
aluguel['banheiro'] = aluguel['banheiro'].str.extract('(.*) Banheiro')

In [27]:
aluguel[cols_str] = aluguel[cols_str].apply(pd.to_numeric, errors='coerce')

In [28]:
aluguel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   aluguel   0 non-null      float64
 1   quartos   156 non-null    float64
 2   endereco  1404 non-null   object 
 3   banheiro  1365 non-null   float64
 4   vaga      0 non-null      float64
 5   area      0 non-null      float64
 6   bairro    1404 non-null   object 
 7   rua       1404 non-null   object 
dtypes: float64(5), object(3)
memory usage: 87.9+ KB


In [29]:
aluguel.describe()

,aluguel,quartos,banheiro,vaga,area
count,0.0,156.0,1365.000000,0.0,0.0
mean,NaN,0.0,2.400000,NaN,NaN
std,NaN,0.0,1.418768,NaN,NaN
min,NaN,0.0,1.000000,NaN,NaN
25%,NaN,0.0,1.000000,NaN,NaN
50%,NaN,0.0,2.000000,NaN,NaN
75%,NaN,0.0,3.000000,NaN,NaN
max,NaN,0.0,8.000000,NaN,NaN


In [30]:
aluguel.shape

(1404, 8)

In [32]:
aluguel = aluguel.fillna(aluguel.median())
aluguel.isnull().sum()

#df.fillna(df.median())

aluguel     1404
quartos        0
endereco       0
banheiro       0
vaga        1404
area        1404
bairro         0
rua            0
dtype: int64